In [7]:
import scanpy as sc
import anndata as ann
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.sparse import issparse
from tqdm import tqdm

from sklearn.decomposition import PCA
import scvelo as scv
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

from unitvelo.eval_utils import cross_boundary_correctness, inner_cluster_coh

In [17]:
import velovi
from velovi import preprocess_data, VELOVI


In [4]:
# import sys
# sys.path.insert(0, '../../../git/VeloVAE/')
# import velovae

# import sys
# sys.path.insert(0, '../../../git/LatentVelo/')
# import latentvelo as ltv

In [13]:
def velovi_pipeline(adata0, name):
    adata = adata0.copy()
    scv.pp.log1p(adata)
    scv.pp.moments(adata, n_pcs=50, n_neighbors=30)
    
    adata = preprocess_data(
        adata,
        spliced_layer = "Ms",
        unspliced_layer = "Mu",
    )
    
    VELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
    vae = VELOVI(adata)
    vae.train()
    
    V = adata.layers['velocity']
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    return V

def velovi_pipeline_without_smoothing(adata0, name):
    adata = adata0.copy()
    scv.pp.log1p(adata)
    scv.pp.neighbors(adata)
    
    adata = preprocess_data(
        adata,
        spliced_layer = "spliced",
        unspliced_layer = "unspliced",
    )
    
    VELOVI.setup_anndata(adata, spliced_layer="spliced", unspliced_layer="unspliced")
    vae = VELOVI(adata)
    vae.train()
    
    V = adata.layers['velocity']
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    return V

In [9]:
## functions used in preparing data for benchmarking

def project_to_pca(adata):
    X = adata.layers['total']
    V = adata.layers['velocity']

    X = np.array(X.A if issparse(X) else X)
    V = np.array(V.A if issparse(V) else V)
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    Y = np.clip(X + V, 0, 1000)


    Xlog = np.log1p(X)
    pca = PCA()
    Xpca = pca.fit_transform(Xlog)

    Ylog = np.log1p(Y)
    Ypca = pca.transform(Ylog)
    V = Ypca - Xpca
    return V

def prepare_for_test(
    adata,
    name,
    func,
    ndims=50,
    pt=True,
):
    x_pca = adata.obsm['X_pca']
    velocity = func(adata, name)

    test = ann.AnnData(X=adata.X, obs=adata.obs, var=adata.var,
                       layers={'total':adata.layers['total'],
                               'velocity':velocity})

    test.obsm['X_pca'] = x_pca[:,:ndims]
    test.obsm['cellrank_baseline'] = adata.obsm['velocity_cr_pca'][:,:ndims]
    if pt:
        test.obsm['pseudotime_baseline'] = adata.obsm['velocity_pst'][:,:ndims]
    else:
        ## this is a lazy implementation, will create meaningless comparison
        ## but it will never get saved
        ## this is just for the maxi dataset that we don't have a good
        ## pseudotime trajectory skeleton for.
        test.obsm['pseudotime_baseline'] = np.zeros_like(test.obsm['cellrank_baseline'])
        
    test.obsm['velocity_pca'] = project_to_pca(test)[:,:ndims]
    
    scv.pp.neighbors(test)
    return test


In [10]:
# the object that will contain the data and data-specific parameters for benchmarking

class BenchMarkingData:
    def __init__(self, name, func, pt=True):
        self.name = name
        adata = sc.read_h5ad(f'../data/benchmarking_data/{name}.h5ad')

        self.adata = prepare_for_test(
            adata,
            name,
            func,
            pt=pt
        )
        
        self.cluster_edges()
        
    def cluster_edges(self):
        if "mini_V3" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('5','14'),
                ('14','8'),
                ('8','21')
            ]
        elif "mini_MN" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('16','15'),
                ('20','23'),
                ('13','18')
            ]
        elif "mini_MD" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('9','12'),
                ('25','4'),
                ('4','6'),
                ('6','22')
            ]
        elif "midi_NM" in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Early_Neural','Neural'),
                ('NMP','Early_Neural'),
                ('NMP','Mesoderm')
            ]
        elif 'midi_Ne' in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Neural','pMN'),
                ('pMN','MN'),
                ('pMN','p3'),
                ('p3','V3')
            ]
        elif 'maxi' in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Early_Neural','Neural'),
                ('NMP','Early_Neural'),
                ('NMP','Mesoderm'),
                ('Neural','pMN'),
                ('pMN','MN'),
                ('pMN','p3'),
                ('p3','V3')
            ]

In [11]:
### the benchmarking functions
### cross_boundary_correctness and inner_cluster_coh use implementation from unitvelo
### web: https://unitvelo.readthedocs.io/en/latest/
### paper: https://www.nature.com/articles/s41467-022-34188-7

def baseline_scores(
    adata
):
    X = adata.obsm['velocity_pca']
    Y1 = adata.obsm['cellrank_baseline']
    Y2 = adata.obsm['pseudotime_baseline']
    cr_scores = np.diagonal(cosine_similarity(X, Y1))
    pt_scores = np.diagonal(cosine_similarity(X, Y2))
    return cr_scores, pt_scores

def run_tests(bm):
    cbd = cross_boundary_correctness(
        bm.adata,
        k_cluster=bm.obs,
        k_velocity='velocity',
        x_emb='X_pca',
        cluster_edges=bm.cluster_edges
    )[1]

    icc = inner_cluster_coh(
        bm.adata,
        k_cluster=bm.obs,
        k_velocity='velocity',
    )[1]
    
    crs, pts = baseline_scores(bm.adata)
    
    return cbd, icc, crs, pts

        

In [12]:
def perform_benchmark(
    pipeline_name,
    velocity_pipeline, 
    output_folder
):
    dataset = ['splicing_mini_V3', 'splicing_mini_MN', 'splicing_mini_MD',
               'splicing_midi_NM', 'splicing_midi_Ne', 'splicing_maxi']
    
    for ds in tqdm(dataset):  
        bm_data = BenchMarkingData(ds, velocity_pipeline, pt=(ds!='splicing_maxi'))
        print(ds)
        cbd, icc, crs, pts = run_tests(bm_data)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_CBD.npy', cbd)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_ICC.npy', icc)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_CRS.npy', crs)
        if ds!='splicing_maxi':
            np.save(f'{output_folder}/{ds}_{pipeline_name}_PTS.npy', pts)

In [ ]:
perform_benchmark(
    pipeline_name='velovi',
    velocity_pipeline=velovi_pipeline, 
    output_folder='../output_data/' 
)

  0%|          | 0/6 [00:00<?, ?it/s]

computing neighbors


In [ ]:
perform_benchmark(
    pipeline_name='velovi_RAW',
    velocity_pipeline=velovi_pipeline_without_smoothing, 
    output_folder='../output_data/'
)